# Dispersions in good compression date

### 2019-04-17 ~ 2019-08-17

In [8]:
from collections import Counter
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
from shutil import copyfile
import glob
from IPython.display import Image
import ipyplot
import datetime
import re


In [9]:
model = "t89"
dispersion_filename = 'output/dispersion_list.csv'
dispersion_day_png_folder = "idl_plots/dispersion_day/*.png"
dispersion_fitting_png_folder = "idl_plots/dispersion/*.png"

In [10]:
dispersion_list = pd.read_csv(dispersion_filename)
dispersion_tplot_list = glob.glob(dispersion_day_png_folder)
dispersion_fitting_plot_list = glob.glob(dispersion_fitting_png_folder)



In [26]:
dispersion_list['datetime'] = dispersion_list.loc[:,'TIME'].apply(datetime.datetime.utcfromtimestamp)

dispersion_list['compression_mode'] = (dispersion_list['datetime'] > pd.Timestamp('2019-4-16')) &(dispersion_list['datetime'] < pd.Timestamp('2019-8-17'))

In [29]:
index = (dispersion_list.loc[:,'rsquare'] > 0.95) & (dispersion_list.loc[:,'direction_et'] == 'outward')  & (dispersion_list['energy'] >= 170) & ((dispersion_list['MLT'] > 18) | (dispersion_list['MLT'] < 6)) & (dispersion_list['compression_mode'])


In [34]:
common_dispersion_list = dispersion_list.loc[index,:]
for i in common_dispersion_list.index:
    display_array = []
    i_toplot = common_dispersion_list.loc[i,:]
    match_date = lambda x: ((datetime.datetime.strptime(i_toplot['datetime_str'], '%Y-%m-%d %H:%M:%S'))
                            >= (datetime.datetime.strptime(re.search("m[0-9]{8}_[0-9]{6}",x).group(),  'm%Y%m%d_%H%M%S'))
                           ) & ((datetime.datetime.strptime(i_toplot['datetime_str'], '%Y-%m-%d %H:%M:%S'))
                                <= (datetime.datetime.strptime(re.search("_[0-9]{8}_[0-9]{6}",x).group(), '_%Y%m%d_%H%M%S'))
                               )
      
    match_list1 = list(map(match_date, dispersion_tplot_list))
    
    if match_list1.count(True) > 0:
        index1 = match_list1.index(True)
        i_tplot1 = dispersion_tplot_list[index1]
        display_array.append(i_tplot1)

    match_list2 = list(map(match_date, dispersion_fitting_plot_list))
    if match_list2.count(True) > 0:
        index2 = match_list2.index(True)
        i_tplot2 = dispersion_fitting_plot_list[index2]
        display_array.append(i_tplot2)

    ipyplot.plot_images(display_array, img_width=430)